In [ ]:
# import the necessary packages
import tensorflow as tf
import numpy as np
import random
import os


# Define parametes

In [ ]:


# model input image size
IMAGE_SIZE = (100,100)

# batch size and the buffer size

BATCH_SIZE = 256
BUFFER_SIZE = BATCH_SIZE * 2

# define autotune
AUTO = tf.data.AUTOTUNE

# define the training parameters
LEARNING_RATE = 0.0001
STEPS_PER_EPOCH = 20
VALIDATION_STEPS = 10
EPOCHS = 5

# define the path to save the model
OUTPUT_PATH = "output"
MODEL_PATH = os.path.join(OUTPUT_PATH, "siamese_network")
OUTPUT_IMAGE_PATH = os.path.join(OUTPUT_PATH, "output_image.png")

epochs = 10
batch_size = 8
margin = 1 

In [ ]:
class MapFunction():
	def __init__(self, imageSize):
		# define the image width and height
		self.imageSize = imageSize
	def decode_and_resize(self, imagePath):
		# read and decode the image path
		image = tf.io.read_file(imagePath)
		image = tf.image.decode_jpeg(image, channels=3)
		# convert the image data type from uint8 to float32 and then resize
		# the image to the set image size
		image = tf.image.convert_image_dtype(image, dtype=tf.float32)
		image = tf.image.resize(image, self.imageSize)
		# return the image
		return image
	def __call__(self, pair, label):
		positive, negative=pair
		positive = self.decode_and_resize(positive)
		negative = self.decode_and_resize(negative)
		return ( positive, negative), label

# PairGenerator

In [ ]:
class PairGenerator:
    def __init__(self, datasetPath):
        self.fruitNames = list()  # path to dir with fruits
        for folderName in os.listdir(datasetPath):
            absoluteFolderName = os.path.join(datasetPath, folderName)
            numImages = len(os.listdir(absoluteFolderName))
            if numImages > 1:
                self.fruitNames.append(absoluteFolderName)
        self.allFruit = self.generate_all_fruit_dict()
    def generate_all_fruit_dict(self):
        allFruit = dict()
        
        for fruitName in self.fruitNames:
            imageNames = os.listdir(fruitName) # all names of photo one fruit
            fruitPhotos = [
                os.path.join(fruitName, imageName) for imageName in imageNames
            ]
            allFruit[fruitName] = fruitPhotos
        return allFruit #all path photo in dict
    def get_next_element(self):
        i=0
        while True:
            i=i+1
                        
            
            imageNames = random.choice(self.fruitNames)
            temporaryNames = self.fruitNames.copy()
            temporaryNames.remove(imageNames)
            negativeNames = random.choice(temporaryNames)

            imagePhoto = random.choice(self.allFruit[imageNames])
            positivePhoto = random.choice(self.allFruit[imageNames])
            negativePhoto = random.choice(self.allFruit[negativeNames])

            yield ((imagePhoto, positivePhoto), 1) 
            yield ((imagePhoto, negativePhoto), 0)
            

            
            

# Creating model

In [ ]:
input = tf.keras.layers.Input(shape=(100, 100, 3))
x = tf.keras.layers.BatchNormalization()(input)
x = tf.keras.layers.Conv2D(4, (5, 5), activation="tanh")(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(16, (5, 5), activation="tanh")(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(256, activation="tanh")(x)
embedding_network = tf.keras.Model(input, x)

input_1 = tf.keras.layers.Input((100, 100, 3))
input_2 = tf.keras.layers.Input((100, 100, 3))

# Calculate Distance

In [ ]:
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.keras.backend.sum(tf.keras.backend.square(x - y), axis=1, keepdims=True)
    return tf.keras.backend.sqrt(tf.keras.backend.maximum(sum_square, tf.keras.backend.epsilon()))

In [ ]:
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = tf.keras.layers.Lambda(euclidean_distance, output_shape=(1,))(
    [tower_1, tower_2]
)
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)

# Define the contrastive loss


In [ ]:
def loss(margin=1):
    """Provides 'contrastive_loss' an enclosing scope with variable 'margin'.

    Arguments:
        margin: Integer, defines the baseline for distance for which pairs
                should be classified as dissimilar. - (default is 1).

    Returns:
        'contrastive_loss' function with data ('margin') attached.
    """

    def contrastive_loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)


        square_pred = tf.keras.backend.square(y_pred)
        margin_square = tf.keras.backend.square(tf.keras.backend.maximum(margin - y_pred, 0))
        return tf.keras.backend.mean((1 - y_true) * square_pred + (y_true) * margin_square)


    return contrastive_loss

In [ ]:
siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
siamese.summary()

In [ ]:
train_path="C:/Users/tokar/OneDrive/Dokumenty/uczelnia/sem6/insert/fruits-360_dataset/fruits-360/Training"

val_path="C:/Users/tokar/OneDrive/Dokumenty/uczelnia/sem6/insert/fruits-360_dataset/fruits-360/Test"

train_dataset=tf.data.Dataset.from_generator(PairGenerator(train_path).get_next_element,
                                        output_signature=((tf.TensorSpec(shape=(), dtype=tf.string),
                                                            tf.TensorSpec(shape=(), dtype=tf.string)),
                                                            tf.TensorSpec(shape=(), dtype=tf.float32)))

val_dataset=tf.data.Dataset.from_generator(PairGenerator(val_path).get_next_element,
                                        output_signature=((tf.TensorSpec(shape=(), dtype=tf.string),
                                                            tf.TensorSpec(shape=(), dtype=tf.string)),
                                                            tf.TensorSpec(shape=(), dtype=tf.float32)))

mapF=MapFunction(IMAGE_SIZE)
train_dataset = train_dataset.map(mapF)
val_dataset = val_dataset.map(mapF)

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(AUTO)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(AUTO)

In [ ]:
history = siamese.fit(
    train_dataset,
    validation_data=val_dataset,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS,
    epochs=10
)